## Convert FAST-STM data (.h5 files) into movies (.mp4)
*NOTE: run cells with shift + enter; (un)comment with ctrl + /*

pyfast documentation: http://fastspm.gitlab.io/pyfast

**1\. Import all relevant packages.**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import io
from shutil import copy2
import base64
from skimage import morphology
import seaborn as sns
sns.set()

import pickle
import pyfastspm as pf

In [ ]:
# do not delete or modify this cell, it is used internally for testing
files = ["20141003_24.h5", "F20190424_1.h5"]

**2\. Specify the files you want to convert.**

Selection of a folder containing .h5 files. Creates a list containing all files, that will be converted.

In [ ]:
# UI All Files in Folder
# Use this box if you want to convert all files in one folder
# If not, skip this box

import tkinter as tk
import glob
from tkinter import filedialog

root  = tk.Tk()
path = filedialog.askdirectory(parent = root, title = "Select the folder with the .h5 files")
root.withdraw()

all_h5_files = path + '/*' + 'h5'
files = glob.glob(all_h5_files)

print(path)
print(files)

Alternatively selection of a single .h5 file that should be converted.

In [ ]:
# UI One File
# Use this box if you want to convert on single file
# If not, skip this box

import tkinter as tk
import glob
from tkinter import filedialog

root  = tk.Tk()

files = filedialog.askopenfilename(parent = root, title = "Select one h5.file file", filetypes = ((".h5 files","*.h5"),("all files","*.*")))
root.withdraw()
files = [files]
print(files)

# **3\. Perform the conversion and export movie.**

**Basic settings:**

_Import_
- ``image_range``: 2-tuple of indices of first and last image to export. If ``None`` is given, exports the full movie. A single image contains two frames up and down.
- ``auto_xphase``: Boolean, finds the correct xphase correction by means of correlation.

_FFT filters_
- ``filter_<>``: Booleans indicating if certain filters should be applied. If ``filter_pump`` is ``True``, ``pump_freqs`` has to be given in advanced settings.
- ``display_spectrum``: Boolean, if ``True`` frequancy domain is displayed.

_Channels to export_
- ``export_channels``: String specifying the channels to export. First part defines y direction: ``'u<>'`` for up, ``'d<>'`` for down or ``'ud<>'`` for both scan directions alternatingly. Second part defines x direction: ``'<>f'`` for forward, ``'<>b'`` for backward or ``'<>i'`` for both scan directions interlaced in each frame.

_Creep correction_
- ``creep_mode``: ``'bezier'``, ``'sin'``, ``'root'`` or ``'None'``. If 'None', no creep correction is used. Else attmepts to correct the creep by either fitting a bezier, sin or root curve. Autofit only performes well for ``i`` channels. For ``f`` or ``b`` channels only use with a known creep (advanced setting ``known_params``, ``known_input``). The bezier reaches a good fit, that is unstable in the case of difficult movies. The sinus fit has a lower quality, but is very stable. The root fit is very fast but need fine tuning.

_Drift correction_
- ``'fft_drift'``: Boolean indicating if drift correction should be applied.
- ``'drifttype'``: ``'full'`` or ``'common'``. If ``'full'`` the movie is showing a square mooving over the sample. The square contains the whole frames. If ``'common'`` the movie shows only the parts of the frame that are captured throughout the movie.
- ``'stepsize'``: ``None`` or integer. An too large stepsize can ignore a small drift, while a too small stepsize can lead to problemes with sub pixel_movements and noise. If ``None`` the defaultstepsize of 40 is used.
- ``'known_drift'``: boolean. If ``'integrated'`` or ``'sequential'`` reads in a .drift.txt file with the same filename as the .h5 file. The shape of the saved array has to be (4, amount of frames) two columns for the integrated path and additional two columns for the sequential path. the keyword decides weather the integrated or the sequancial columns are used for the drift path.

_Streak removal for interlacing_
- ``remove_streaks``: Boolean indicating if streaks, which often occur in ``i`` mode, should be removed by applying a convolutional filter in y direction. Has no effect for not interlaced channels.

_Export_
- ``export_movie``: Boolean indicating if movie should be exported as ``.mp4`` video.
- ``export_frames``: Boolean indicating if a single frame or a range of frames should be exported.
- ``frame_export_images``: Either
 - 1 element list giving the index of the frame to export.
 - 2 element list giving the indices of the first and last frame to export. Has no effect if ``export_movie`` is ``False``.
- ``frame_export_channel``: Channel of frames to be exported. If ``export_channels`` is ``'u<>'`` or ``'d<>'``, has to be equal to ``export_channels``. If ``export_channels`` is ``'ud<>'``, has to be either ``'u<>'`` or ``'d<>'``. Has no effect if ``export_movie`` is ``False``.



**Advanced settings:**

_Import and phase_
- ``x_phase_correction``: Occasionally x phase in metadata is off by 1. In interlacing mode, this leads to features appearing fringed or blurred in x direction. In this case, try setting this parameter to ``-1`` or ``1``. Should be left at ``0`` otherwise.
- ``y_phase``: y phase to correct for shaky movies in ``ud`` channels. For ``i`` channels, shaky movies should be corrected by creep correction and y_phase should be set to 0. For ``f`` and ``b`` channels, y_phase should be set manually (``-1`` seems to work in the old STM lab).

_FFT filters_
- ``filter_broadness``: Broadness of fft filters which cut out single frequencies in Hz. If ``None`` is given, takes the y scan frequency in the metadata.
- ``num_x_overtones``: Number of x overtones to filter out. Has no effect if ``filter_x_overtones`` is ``False``.
- ``high_pass_params``: Frequency and sigma for high pass filter. Has no effect if ``filter_high_pass`` is ``False``.
- ``num_pump_overtones``: Number of pump frequency overtones to filter out. Has no effect if ``filter_pump`` is ``False``.
- ``pump_freqs``: Tuple of pump frequencies. Has no effect if ``filter_pump`` is ``False``.
- ``fft_display_range``: 2-tuple of ints or float specifying the range in Hz in which the spectrum should be displayed, if ``display_spectrum`` is ``True``.

_Bezier creep correction_
- ``weight_boundary``: *Additional* weighting of pixels at upper and lower boundary of each frame in automated creep correction. Has no effect if ``correct_creep`` is ``False``, ``creep_known`` is ``True`` or ``export_channels`` does not contain ``i``.
- ``creep_num_cols``: Number of columns for fitting of creep correction.
- ``known_input``: ``None`` or 3-tuple of the form ``(shape[0], shape[1], shape[2])``. Known creep parameters from either a previous run or manual fitting. If ``None`` parameters are fitted.

_Other creep correction_
- ``initial_guess``: Tuple. Set value for initial gues of creep function parameters (phase for sin, controlls curvature).
- ``guess_ind``: Value determining how much of each frame is affected by creep (only applicable for sin). Values >= 1 are interpreted as lines of the frame, values between 0 and 1 will be interpreted as relative parts of the frames.
- ``known_params``: Tuple. The known creep function parameters either from previous fit or manual determination. If this is not none, no fit for the creep parameters will be perfomed.

_Export_
- ``export_drift_matrix``: exports the .txt file which can be modified to manually change the drift correction.
- ``import_drift_matrix``: imports the modified drift_matrix in order to bypass the autocorrelation of the drift correction.
- ``contrast``: 2-tuple of floats 0<=x<=1. Defines where in the data histogram of the *whole* movie the contrast is cut.
- ``fps_factor``: Playback speed of exported movie compared to real time. Has no effect if ``export_movie`` is ``False``.
- ``color_map``: Name of color map for movie/frame export.
- ``frame_export_format``: Image format for frame export. Has no effect if ``export_frames`` is ``False``.
- ``auto_label``: boolean. Adds frame number and time to the movie.

In [ ]:
### Basic settings
#with open f:
s = {
    ### Import
    'image_range'           : None, #(0, 553), #(0,97), # (100,750), #(200,2000), # None
    'auto_xphase'           : True,

    ### FFT filters
    'filter_x'              : True,
    'filter_y'              : True,
    'filter_x_overtones'    : True,
    'filter_high_pass'      : True,
    'filter_pump'           : True, # if True, pump_freqs has to be given in advanced settings
    'filter_noise'          : False,
    'display_spectrum'      : True,
    
    ### Channels to export
    'export_channels'       : 'udi',
    
    ### Creep correction
    'creep_mode'            : 'sin', # 'sin', 'bezier', 'root', 'None'

    ### Drift correction
    'fft_drift'             : True,
    'drifttype'             : 'full', # 'full' or 'common'
    'stepsize'              : 10,   # 'set to None for default'
    'known_drift'           : False, # 'integrated', 'sequential', False

    
    ### Streak removal for interlacing
    'remove_streaks'        : False,
    
    # Export
    'export_movie'          : True,
    'export_frames'         : False,
    'frame_export_images'   : (0,1),
    'frame_export_channel'  : 'udi', # 'u<>' or 'd<>' not 'ud<>'
}

### Advanced settings
s.update({
    ### Import and phase
    'x_phase_correction'    : 0,  # applies phase correction after autophase
    'y_phase'               : 0,  # enter adapted phase before auto correction 
    'x_phase'               : 0,
    'sigma_gauss'           : 0,
    
    ### FFT filters
    'filter_broadness'      : None,
    'num_x_overtones'       : 10,
    'high_pass_params'      : (1000., 600.),
    'num_pump_overtones'    : 3,
    'pump_freqs'            : tuple((1500.,1000.,)), # Pump frequency has to be found manually, e.g. with pf.show_fft(ft, <some_range>)
    'fft_display_range'     : (0, 40000),
    
    ### Creep correction Bezier
    'weight_boundary'       : 0.0,
    'creep_num_cols'        : 3,
    'known_input'           : None, # None or Tuple in Form (0.5000, 0.3463, 0.4676)
    
    ### Creep correction non Bezier
    'initial_guess'         : (0.6,),   # example for 'sin'
    #'initial_guess'         : (1, -1), # example for 'root'
    'guess_ind'             : 0.3,
    'known_params'          : None, #None or float
    
    ### Export
    'contrast'              : (0.01, 0.99),
    'scaling'               : (2.,2.), # changed from (4.,4.)
    'fps_factor'            : 5,
    'color_map'             : 'inferno',
    'frame_export_format'   : 'png',
    'auto_label'            : True,
})


for i, file in enumerate(files):
    
    ################################################################
    ### --------------------- Init stuff ----------------------- ###
    ################################################################
    
    ### Create local variables out of dictionary entries
    for k,v in s.items():
        if type(v) is list:
            locals()[k]=v[i]
        else:
            locals()[k]=v
    
    ### Handle some errors which users might make
    frame_export_channel = pf.error_catcher(export_frames, 
                                            frame_export_channel,
                                            export_channels,
                                            image_range,
                                            frame_export_images)
    

    ################################################################
    ### ------------- Load file and correct phase -------------- ###
    ################################################################
    
    
    ### Load hdf5 file   
    ft = pf.FastMovie(str(file), y_phase=y_phase) 
    

    ### Occasionally x phase in metadata is off
    x_phase = ft.correct_phase(0, sigma_gauss=sigma_gauss, manual_x=x_phase_correction, manual_y=y_phase)


    ################################################################
    ### --------------------- fft filter ----------------------- ###
    ################################################################
    
    filterparam = [filter_x, filter_y, filter_x_overtones, filter_high_pass, filter_pump, filter_noise, display_spectrum]
    if any(filterparam):
    
        pf.filter_movie(ft,
                        filterparam,
                        filter_broadness,
                        fft_display_range,
                        pump_freqs,
                        num_pump_overtones,
                        num_x_overtones,
                        high_pass_params)
    

    ft.reshape_to_movie(channels=export_channels)
 
    ################################################################
    ### -------------------- Creep Correction ------------------ ###
    ################################################################
    

    if creep_mode == 'bezier':
        creep = pf.Creep(ft, index_to_linear=guess_ind)
        opt, grid = creep.fit_creep_bez(col_inds=np.linspace(ft.data.shape[2]*.25, ft.data.shape[2]*.75, creep_num_cols).astype(int)
                                       ,w=weight_boundary, known_input=known_input)

    elif creep_mode != 'bezier' and creep_mode != 'None':
        #creep = pf.Creep(ft, index_to_linear=guess_ind)
        #grid = creep.fit_creep_sin(creep.sin_one_param,(initial_guess_phase, guess_ind),
        #                           known_params = known_params)
        creep = pf.Creep(ft, index_to_linear=guess_ind, creep_mode=creep_mode)
        grid = creep.fit_creep(initial_guess, #### Add params here!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
                               known_params = known_params)
    else:
        grid = None
        
        

    interpolation_matrix_up, interpolation_matrix_down = pf.interpolate(ft, 
                                              grid=grid,
                                              give_grid=True)


    if export_movie or export_frames:
        imrange = image_range

    
    ################################################################
    ### --------------------- Interpolation -------------------- ###
    ################################################################
    
        
    pf.interpolate(ft, 
                   grid=grid, 
                   image_range=imrange,
                   interpolation_matrix_up=interpolation_matrix_up,
                   interpolation_matrix_down=interpolation_matrix_down)
        

    if remove_streaks:
        edge_removal = [[-1./12.], [3./12.], [8./12.], [3./12.], [-1./12.]]
        pf.conv_mat(ft, edge_removal, image_range=imrange)

        
    ################################################################
    ###------------------------- Drift --------------------------###
    ################################################################

    if fft_drift:
        driftrem = pf.Drift(ft, stepsize=stepsize, boxcar=True)
        ft.data, drift_path = driftrem.correct(drifttype,
                                               known_drift=known_drift)

    if image_range == None:
        first = 0
        last = -1
    elif 'u' and 'd' in export_channels:
        first = image_range[0]*2
        last = image_range[1]*2
    else:
        first = image_range[0]
        last = image_range[1]
    
    
    if fft_drift:
        a = np.loadtxt(file.replace(".h5", ".drift.txt")).T[0:2, :]
        cols=3
    else:
        cols=2
    fig, ax = plt.subplots(nrows=1, ncols=cols, figsize = (12,4))
    ax[0].imshow(ft.data[first,:,:], aspect ='auto')
    ax[1].imshow(ft.data[last,:,:], aspect ='auto')
    if fft_drift:
        ax[2].plot(a[0,first:last])
        ax[2].plot(a[1,first:last])
        ax[2].legend(['y','x'])
        ax[2].set_xlabel('frames')
        ax[2].set_ylabel('pixel')
    ax[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    ax[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    plt.tight_layout()
    plt.show()
    

    ################################################################
    ### ----------------------- Export ------------------------- ###
    ################################################################
        
    ### Export movie / frame and close file
    if export_movie:
        ft.export_movie(color_map=color_map, contrast=contrast, fps_factor=fps_factor,
                        image_range=image_range,scaling=scaling, auto_label=auto_label) # writes .mp4 file into same folder the .h5 file was loaded from
        if image_range == None:
            pick_data=ft.data
        else:
            pick_data=ft.data[image_range[0]*2:image_range[1]*2] # factor 2 due to two frames per image in udi
        pickle.dump([pick_data,ft.fps,image_range], open(str(file)+".p", "wb" )) 
        with open (str(file)+'_meta.txt','w') as f:
            f.write('Metadata:\n' + str(ft.metadata) +'\nExport Options:\n'+str(s))
    if export_frames:
        ft.export_frame(images=frame_export_images, channel=frame_export_channel, color_map=color_map,
                        file_format=frame_export_format, contrast=contrast)
    

    
ft.close()